# Free Energy Calculations: Methane in Water

This is a tutorial based on [this](http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/free_energy/index.html) one. We will use BAC2.0

The system contains a single molecule of methane (called "ALAB" in the coordinate file, for the β-carbon of alanine) in a box of 596 TIP3P water molecules.

In [1]:
import bac.simulate.gromacs as gmx

In [2]:
from pathlib import Path

In [3]:
em_steep = gmx.Simulation(integrator='steep', number_of_steps=2500)

em_steep.name = 'em_steep'
em_steep.coordinates = Path('input_files/methane_water.gro').absolute()
em_steep.topology = Path('input_files/topol.top').absolute()

em_steep.energy_log_output_frequency = 1
em_steep.energy_output_frequency = 100

In [4]:
em_steep.delta_time = 0.002

In [5]:
em_steep.center_of_mass_motion = gmx.simulation_controller.CenterOfMassMotion.linear
em_steep.com_motion_removal_frequency = 100

In [6]:
em_steep.generate_velocities = False

em_steep.minimization_tolerance = 100
em_steep.minimization_step_size = 0.01
em_steep.minimization_correction_steps = 10
em_steep.minimization_steepest_descent_frequency = 1000

In [7]:
nbc = em_steep.non_bonded_controller
nbc.cutoff_scheme = gmx.CutoffSchemeType.verlet
nbc.neighbor_list_update_frequency = 10
nbc.neighbor_list_type = gmx.NeighborListType.grid
nbc.periodic_boundary_condition = gmx.PeriodicBoundaryConditionType.xyz
nbc.periodic_molecules = False
nbc.verlet_buffer_tolerance = 0.005
nbc.cutoff = 1.2

nbc.coulomb_type = gmx.CoulombType.pme
nbc.coulomb_modifier = gmx.CoulombModifierType.potential_shift_verlet
nbc.coulomb_switch_cutoff = 0.0
nbc.coulomb_cutoff = 1.2


nbc.van_der_waals_type = gmx.VanDerWaalsType.cutoff
nbc.van_der_waals_modifier = gmx.VanDerWaalsModifierType.potential_shift
nbc.van_der_waals_switch_cutoff = 1.0
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = gmx.LongRangeDispersionCorrectionType.energy_and_pressure

nbc.fourier_spacing = 0.12
nbc.pme_order = 6
nbc.ewald_tolerance_coulomb = 1e-06
nbc.ewald_epsilon_surface = 0

In [8]:
gmx.non_bonded_controller.NonBondedController.coulomb_switch_cutoff

In [9]:
fe = gmx.FreeEnergyController()
em_steep.free_energy_controller = fe


# fe.delta_lambda = 0
# fe.calculate_lambda_neighbors = 1

# This will be set by an ensemble_modifier is the workflow has one. There is no need to do it now.
# fe.initial_lambda_state = 0

fe.fep_lambdas = [0]*21
fe.van_der_waals_lambdas = [x*0.05 for x in range(21)]

fe.soft_core_alpha = 0.5
fe.soft_core_coulomb = False
fe.soft_core_power = 1
fe.soft_core_sigma = 0.3

fe.couple_molecule_groups = 'Methane'
fe.couple_type_initial_lambda = 'vdw'
fe.couple_type_final_lambda = 'none'
fe.couple_intramolecular = False

fe.output_frequency = 10

In [10]:
em_steep.constraints.continuation = False
em_steep.constraints.type = gmx.ConstraintType.h_bonds
em_steep.constraints.algorithm = gmx.ConstraintAlgorithmType.lincs
em_steep.constraints.lincs_order = 12

Next energy minimization step

In [11]:
em_bfgs = next(em_steep)

In [12]:
em_bfgs.integrator = gmx.Integrator.l_bfgs
em_bfgs.constraints.type = gmx.ConstraintType.none
em_bfgs.name = 'em_bfgs'

In [13]:
nvt = next(em_bfgs)
nvt.name = 'nvt'

In [14]:
nvt.integrator = 'sd'
nvt.delta_time = 0.002
nvt.number_of_steps = 50000
nvt.com_motion_removal_frequency = 100

nvt.generate_velocities = True
nvt.generate_temperature = 300

In [15]:
nvt.temperature_controller.groups = 'system'
nvt.temperature_controller.time = 1
nvt.temperature_controller.temperature = 300

## Simulation and Workflow

In [16]:
from bac.simulate import Workflow, Replica, LambdaWindow

In [17]:
wf = Workflow(resource='MPB', name='MeOH_H2O')

In [18]:
wf.add_simulation(em_steep)
wf.add_simulation(em_bfgs)
wf.add_simulation(nvt)

In [19]:
wf.ensembles = [Replica(number_of=5), LambdaWindow(number_of_states=5)]

In [20]:
# wf.preprocess_simulations(execute=False)

In [21]:
# wf.generic_executable()

In [22]:
wf.write_generic_bash_executable(path=wf.path)